In [ ]:
from array import array 
from pathlib import Path 
import oci 
from multiprocessing import Process 
from multiprocessing import Semaphore 

In [ ]:
# Number of max processes allowed at a time 
concurrency= 5 
sema = Semaphore(concurrency) 
# The root directory path, Replace with your path 
p = Path('/home/datascience/vulcan_2_0/user_2/data/img_packs/') 
# The Compartment OCID 
compartment_id = "ocid1.compartment.oc1..aaaaaaaa334d2c5yz5agpeifnxs5kxbdw2ywsizjjjgddzp5edrxweassqya" 
# The Bucket name where we will upload 
bucket_name = "cairo_images" 

In [ ]:
def upload_to_object_storage(path:str,name:str,object_storage_client,namespace): 
  with open(path, "rb") as in_file: 
    print("Starting upload {}", format(name)) 
    object_storage_client.put_object(namespace,bucket_name,name,in_file) 
    print("Finished uploading {}".format(name)) 
    sema.release() 

In [ ]:
def createUploadProcess(object:Path,object_storage_client,namespace,proc_list): 
  name = object.relative_to(p).as_posix() 
  sema.acquire() 
  process = Process(target=upload_to_object_storage, args=(object.as_posix(),name,object_storage_client,namespace)) 
  proc_list.append(process) 
  process.start() 

In [ ]:
def createUploadProcess(object:Path,object_storage_client,namespace,proc_list): 
  name = object.relative_to(p).as_posix() 
  sema.acquire() 
  process = Process(target=upload_to_object_storage, args=(object.as_posix(),name,object_storage_client,namespace)) 
  proc_list.append(process) 
  process.start() 

In [ ]:
def processDirectoryObjects(object:Path,object_storage_client,namespace,proc_list): 
  if object.is_file(): 
    createUploadProcess(object,object_storage_client,namespace,proc_list) 

In [ ]:
def processDirectory(path:Path,object_storage_client,namespace,proc_list): 
  if path.exists(): 
    print("in directory ---- " + path.relative_to(p).as_posix()) 
    for objects in path.iterdir(): 
      if objects.is_dir(): 
        processDirectory(objects,object_storage_client,namespace,proc_list) 
      else: 
        processDirectoryObjects(objects,object_storage_client,namespace,proc_list) 

In [ ]:
config = oci.config.from_file() 
object_storage_client = oci.object_storage.ObjectStorageClient(config) 
namespace = object_storage_client.get_namespace().data 
proc_list: array = [] 
sema = Semaphore(concurrency) 
if p.exists() and p.is_dir(): 
    processDirectory(p,object_storage_client,namespace,proc_list) 
for job in proc_list: 
    job.join()